In [1]:
#dependencies 
#had to 
import pandas as pd 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
from coincallandwritepythonfile import refined_coin_list

In [2]:
refined_coin_list

,id,name,symbol,rank,File_Name,injest_datetime
1,1,Bitcoin,BTC,1,Coin_Ranking_Dim,04/20/2022 15:45:17
2,1027,Ethereum,ETH,2,Coin_Ranking_Dim,04/20/2022 15:45:17
3,825,Tether,USDT,3,Coin_Ranking_Dim,04/20/2022 15:45:17
4,1839,BNB,BNB,4,Coin_Ranking_Dim,04/20/2022 15:45:17
5,3408,USD Coin,USDC,5,Coin_Ranking_Dim,04/20/2022 15:45:17
...,...,...,...,...,...,...
97,2586,Synthetix,SNX,97,Coin_Ranking_Dim,04/20/2022 15:45:17
98,4846,Kava,KAVA,98,Coin_Ranking_Dim,04/20/2022 15:45:17
99,9444,Kyber Network Crystal v2,KNC,99,Coin_Ranking_Dim,04/20/2022 15:45:17
100,2777,IoTeX,IOTX,100,Coin_Ranking_Dim,04/20/2022 15:45:17


In [74]:
#get ids for each coin and convert to list
coin_ids = refined_coin_list.id.to_list()
type(coin_ids)
#getting 5 first ids to try in practice run
five_ids_to_try = coin_ids[:4]

In [128]:
def get_coin_info(list_of_coins_, api_key):
    df = pd.DataFrame()
    count =0
    for id in list_of_coins_:
        url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
        parameters = {
            'id': id
            }
        headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key
        }

        session = Session()
        session.headers.update(headers)
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        #navigate json
        #read to DF to see what we have
        cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
        #need to explode this, then re join 
        #expand nest columns
        expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')
        #showing USD as an index column, going to drop
        count +=1
        expanded_df.reset_index(drop = True, inplace= True)
               
        trial = cmc_df.drop(['quote','tags'], axis= 1)
        #had to reset index or set it manually to match the expanded DF 
    
        trial.reset_index(drop = True, inplace= True)
        #trial.reset_index(drop = True, inplace= True)
        
        #had to add 
        coin_price_df = pd.concat([trial,expanded_df], axis=1,  )
        #coin_price_df =coin_price_df.set_index(pd.Index([count]), inplace= True)

        #going to try to append this to another dataframe
        #df = pd.DataFrame()
        coin_price_df.set_index(pd.Index([count]),inplace= True)

        df = df.append(coin_price_df)
    return df

        


In [129]:
df = get_coin_info(five_ids_to_try, config.coin_api_key)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [119]:
df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1,Bitcoin,BTC,bitcoin,9340,2013-04-28T00:00:00.000Z,21000000,1.901733e+07,1.901733e+07,1,...,0.192251,0.393836,0.679499,1.240433,3.762888,1.469869,7.890575e+11,41.1734,8.713214e+11,2022-04-21T00:32:00.000Z
0,1027,Ethereum,ETH,ethereum,5652,2015-08-07T00:00:00.000Z,None,1.204620e+08,1.204620e+08,1,...,-0.075053,0.141678,-1.289393,6.329130,12.158684,2.042204,3.709063e+11,19.3541,3.709063e+11,2022-04-21T00:31:00.000Z
0,825,Tether,USDT,tether,32070,2015-02-25T00:00:00.000Z,None,8.299786e+10,8.561395e+10,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",...,-0.005693,-0.006766,-0.011284,-0.014243,-0.025391,-0.012529,8.301465e+10,4.3319,8.563127e+10,2022-04-21T00:31:00.000Z
0,1839,BNB,BNB,bnb,802,2017-07-25T00:00:00.000Z,165116760,1.632770e+08,1.632770e+08,1,...,0.345191,0.337362,-1.038769,6.098762,5.742056,-5.330557,6.859937e+10,3.5794,6.937233e+10,2022-04-21T00:31:00.000Z


In [103]:
#https://coinmarketcap.com/api/documentation/v1/#operation/getV2CryptocurrencyQuotesLatest
#now need to reference the coin ids and pull in data from the api accordinly 

config.coin_api_key

url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
parameters = {
    'id': [1]
    }
headers = {
'Accepts': 'application/json',
'X-CMC_PRO_API_KEY': config.coin_api_key
}

session = Session()
session.headers.update(headers)
response = session.get(url, params=parameters)
data = json.loads(response.text)


In [104]:
#navigate json
data['data']
#read to DF to see what we have
cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
#need to explode this, then re join 
cmc_df['quote'][0]

{'USD': {'price': 41451.1353306875,
  'volume_24h': 27912601142.293224,
  'volume_change_24h': 11.804,
  'percent_change_1h': 0.09411368,
  'percent_change_24h': -0.11532505,
  'percent_change_7d': 0.78096388,
  'percent_change_30d': 1.10388286,
  'percent_change_60d': 3.42483063,
  'percent_change_90d': 1.74784875,
  'market_cap': 788289960909.4786,
  'market_cap_dominance': 41.0747,
  'fully_diluted_market_cap': 870473841944.44,
  'last_updated': '2022-04-21T00:20:00.000Z'}}

In [97]:
#expand nest columns
expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')
#showing USD as an index column, going to drop
expanded_df.reset_index(drop = True, inplace= True)
#can double check shape
expanded_df.shape
trial = cmc_df.drop(['quote','tags'], axis= 1)
#had to reset index or set it manually to match the expanded DF 
trial.set_index(pd.Index([0]), inplace= True)
#trial.reset_index(drop = True, inplace= True)

coin_price_df = pd.concat([trial,expanded_df], axis=1 )
coin_price_df.set_index(pd.Index([22]), inplace= True)

#going to try to append this to another dataframe
df = pd.DataFrame()

df = df.append(coin_price_df)
df.head()

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
22,1,Bitcoin,BTC,bitcoin,9340,2013-04-28T00:00:00.000Z,21000000,19017287,19017287,1,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [52]:
df

""


In [48]:
coin_price_df

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,1,Bitcoin,BTC,bitcoin,9340,2013-04-28T00:00:00.000Z,21000000,19017287,19017287,1,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [25]:
trial = cmc_df.drop(['quote','tags'], axis= 1)
trial.shape

(1, 16)

In [27]:
coin_price_df

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
1,1.0,Bitcoin,BTC,bitcoin,9340.0,2013-04-28T00:00:00.000Z,21000000.0,19017287.0,19017287.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.208146,0.288073,0.536021,0.829391,3.857871,0.565335,7.901044e+11,41.1164,8.724794e+11,2022-04-20T22:44:00.000Z


In [26]:
768620549906.7964/ 40426.70598566949

19012693.0

In [14]:
trial.index

Index(['USD'], dtype='object')